In [ ]:
from local.imports import *
from local.test import *
from local.core import *
from local.layers import *
from local.data.all import *
from local.notebook.showdoc import show_doc
from local.optimizer import *
from local.learner import *
from local.text.data import *
from local.text.models.core import *
from local.text.models.awdlstm import *
from local.callback.rnn import *
from local.callback.all import *

# Integration test on Wikitext-2

> Training a Language Model on WT2

In [ ]:
path = untar_data(URLs.WIKITEXT_TINY)

In [ ]:
def istitle(line):
    return len(re.findall(r'^ = [^=]* = $', line)) != 0

def read_file(filename):
    articles = []
    with open(filename, encoding='utf8') as f:
        lines = f.readlines()
    current_article = ''
    for i,line in enumerate(lines):
        current_article += line
        if i < len(lines)-2 and lines[i+1] == ' \n' and istitle(lines[i+2]):
            articles.append(current_article)
            current_article = ''
    articles.append(current_article)
    return np.array(articles)

In [ ]:
train = read_file(path/'train.txt')
valid = read_file(path/'valid.txt')

In [ ]:
all_texts = L(np.concatenate([valid, train]), use_list=True)

In [ ]:
count = Counter([p for t in all_texts for p in t.split()])
vocab = make_vocab(count)

In [ ]:
splits = ([list(range(60, len(all_texts))), list(range(60))])
dsrc = DataSource(all_texts, type_tfms=[Numericalize(vocab)], filts=splits)

In [ ]:
bs,sl = 104,72
train_ds = LM_Dataset(dsrc.train, bs=bs, seq_len=sl, shuffle=True)
train_samp = LM_Sampler(train_ds)
train_dl = TfmdDL(train_ds, bs=bs, sampler=train_samp, num_workers=2, tfms=Cuda())
valid_ds = LM_Dataset(dsrc.valid, bs=bs, seq_len=sl, shuffle=False)
valid_samp = LM_Sampler(valid_ds)
valid_dl = TfmdDL(valid_ds, bs=2*bs, sampler=valid_samp, num_workers=2, tfms=Cuda())
dbch = DataBunch(train_dl, valid_dl)
dbch.show_batch()

index,text
0,"xxunk xxunk = Dover Athletic F.C. = xxunk xxunk Dover Athletic Football Club is a professional association football club based in the town of Dover , Kent , England . The club was formed in 1983 after the dissolution of the town 's previous club , Dover F.C. , whose place in the Southern League was taken by the new club . In the 1989 – 90 season Dover Athletic won the"
1,"of four new dreadnoughts , SMS <unk> <unk> , that made up the Tegetthoff class — the only dreadnoughts built for the Austro @-@ Hungarian Navy — came into active service . With the commissioning of these dreadnoughts , Zrínyi and her sisters were moved from the 1st Division to the 2nd Division of the 1st Battle Squadron . xxunk xxunk = = = World War I = = = xxunk xxunk"
2,"AADT ) , a measure of traffic volume for any average day of the year . Only the N @-@ 71 / N @-@ 88 concurrency is part of the National Highway System ( NHS ) , a network of highways identified as being most important for the economy , mobility and defense of the nation . xxunk xxunk = = History = = xxunk xxunk N @-@ 88 was unofficially designated"
3,"<unk> Bank buoy which directed shipping into the canal . xxunk Commander Alfred Godsal led the assault in HMS Brilliant and it was he who stumbled into the most effective German counter @-@ measure first . As Brilliant <unk> through the <unk> , the lookout spotted the buoy ahead and Godsal headed directly for it , coming under even heavier fire as he did so . Passing the navigation marker at speed"
4,"that this implies that the digital revolution is dominated by people "" on <unk> who have no furniture "" . Similarly , Underwood and his associates are <unk> clad , Barnes shows a lack of fashion recognition . xxunk xxunk = = = Release = = = xxunk xxunk The episode was broadcast online by Netflix on February 1 , 2013 as part of the simultaneous release of all 13 episodes of"
5,"<unk> Brian <unk> , and a third exclusively for filming airborne sequences . This expansion increased the number of APF crews and stages to five each . A typical episode contained around 100 effects shots ; Meddings ' team completed up to 18 per day . xxunk An addition to the effects department was Mike <unk> , who served as Meddings ' assistant in designing vehicles and buildings . Meddings and <unk>"
6,"withdraw in contact , taking their wounded with them ; the action was later rewarded with a Military Cross for the platoon commander , who was badly wounded after taking over one of the machine guns after its crew had been wounded . xxunk Near the Sungei Murai ( Murai River ) , 15 Platoon — under Lieutenant John Meiklejohn — had been stationed in support of a company from the 2"
7,"which is also known as the Golden Temple , was built in <unk> by Queen <unk> <unk> of <unk> . The two pinnacles of the temple are covered in gold and were donated in 1839 by <unk> Singh , the ruler of Punjab . The dome is scheduled to receive gold plating through a proposed initiative of the Ministry of Culture and Religious Affairs of Uttar Pradesh . Numerous rituals , prayers"
8,"mainland Europe by rising sea levels before the last ice age had completely finished , it has fewer land animal and plant species than Great Britain , which separated later , or mainland Europe . There are 55 mammal species in Ireland and of them only 26 land mammal species are considered native to Ireland . Some species , such as , the red fox , <unk> and <unk> , are very"
9,"shear caused significant weakening thereafter . On October 1 , Polo regained tropical storm strength as it was crossing into the north Central Pacific basin . It dissipated as a tropical cyclone later that day . There were no casualties or damages caused by Polo . xxunk xxunk = = = Tropical Storm Simon = = = xxunk xxunk A weakly defined tropical wave moved off the northwest coast of Africa on"


In [ ]:
config = awd_lstm_lm_config.copy()
config.update({'input_p': 0.6, 'output_p': 0.4, 'weight_p': 0.5, 'embed_p': 0.1, 'hidden_p': 0.2})
model = get_language_model(AWD_LSTM, len(vocab), config=config)

In [ ]:
opt_func = partial(Adam, wd=0.1, eps=1e-7)
cb_funcs = [partial(MixedPrecision, clip=0.1), partial(RNNTrainer, alpha=3, beta=2)]

In [ ]:
learn = Learner(model, dbch, loss_func=CrossEntropyLossFlat(), opt_func=opt_func, cb_funcs=cb_funcs)

In [ ]:
learn.fit_one_cycle(10, 5e-3, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,time
0,7.701850,6.290051,01:19
1,6.243032,5.507934,01:21
